In [1]:
import tempfile
import pandas as pd
import numpy as np

import os
import shutil

from deriva_common import HatracStore, ErmrestCatalog, datasets, get_credential

import IPython.core.debugger

In [ ]:
dbg = IPython.core.debugger.Pdb
from IPython.core.debugger import set_trace

In [62]:
# Configuring the logger for debug level will display the uri's generated by the api
debug = False
if debug:
    import logging
    logger = logging.getLogger('deriva_common.datasets')
    logger.setLevel(logging.DEBUG)
    ch = logging.StreamHandler()
    ch.setLevel(logging.DEBUG)
    logger.addHandler(ch)

In [2]:
def get_synapses(synapsefiles, study):
   
    '''
     synapsefile: a HatracStore object
     study: a dictionary that has URLs for the two images, before and after
     
     returns two pandas that have the synapses in them.
     ''' 

    try:
        # Get a path for a tempory file to store HATRAC results 
        path = os.path.join(tempfile.mkdtemp(), 'image')
        
        # Get the before image from hatrac, be careful in case its missing
        if study['Region 1 URL']:
            synapsefiles.get_obj(study['Region 1 URL'], destfilename=path)
            img1 = pd.read_csv(path)
            img1.drop(img1.index[0],inplace=True)
        else:
            img1 = None
        
        # Get the after image from hatrac, be careful in case its missing
        if study['Region 2 URL']:
            synapsefiles.get_obj(study['Region 2 URL'], destfilename=path)
            img2 = pd.read_csv(path)
            img2.drop(img2.index[0],inplace=True)
        else:
            img2 = None
    finally:
        shutil.rmtree(os.path.dirname(path))
    return {'before':img1, 'after': img2, 'type': study['type']}    

In [3]:
# Plot a study

import plotly as py
import plotly.graph_objs as go

py.offline.init_notebook_mode(connected=True)

def plot_study(study):
    '''
    Create a 3D scatter plot of a study.
    '''
    trace1 = go.Scatter3d(
        x=study['before']['X'],
        y=study['before']['Y'],
        z=study['before']['Z'],
        mode='markers',
        marker=dict(
            size=2,
            line=dict(
            color='rgb(0,255,0)',
            width=0.5
            ),
        opacity=0.8
        )
    )

    trace2 = go.Scatter3d(
        x=study['after']['X'],
        y=study['after']['Y'],
        z=study['after']['Z'],
        mode='markers',
        marker=dict(
            size=2,
            line=dict(
                color='rgb(255,0,0)',
                width=.5
            ),
        opacity=0.8
        )
    )

    data = [trace1, trace2]

    layout = go.Layout(
        margin=dict(
            l=0,
            r=0,
            b=0,
            t=0
        )
    )

    fig = go.Figure(data=data, layout=layout)
    py.offline.iplot(fig)

In [5]:
# Need to use Deriva authentication agent before executing this
credential = get_credential("synapse.isrd.isi.edu")

objectstore = HatracStore('https','synapse.isrd.isi.edu',credentials=credential)
catalog = ErmrestCatalog('https','synapse.isrd.isi.edu', 1, credentials=credential)

In [77]:
# Get set of studies as a list from the Synaptic Pair Study table. 

ds = datasets.from_catalog(catalog)

# Get a couple of useful tables....
behaviors = ds.Zebrafish.Behavior
pairs = ds.Zebrafish.tables['Image Pair Study']
studies = ds.Zebrafish.tables['Synaptic Pair Study']
images = ds.Zebrafish.Image

protocol_steps = ds.Synapse.tables['Protocol Step']

B = behaviors.as_('B')
S = studies.as_('S')
I = images.as_('I')
P = pairs.as_('P')

# This doesn't work....
#foo = S \
#    .join(P) \
#    .join(I) \
#    .join(B, on=(I.Subject == B.Subject)) \
#    .select(S.Study, \
#            I.Subject, \
#            B.columns['Learned?'], 
#            S.columns['Region 1 URL'], \
#            S.columns['Region 2 URL'])

# Image->Protocol Step[ID]
study_query = S \
    .join(P) \
    .join(I) \
    .join(B, on=(I.Subject == behaviors.Subject)) \
    .join(protocol_steps, on=(I.Step == protocol_steps.ID)) \
    .select(S.Study, \
            I.Subject, \
            B.columns['Learned?'], 
            S.columns['Region 1 URL'], \
            S.columns['Region 2 URL'])
          
study_list = list(study_query)

# Hack in study type, this should be replaced with a join in the previous expression.
for i in study_list:
    i['type'] = 'control'


/attribute/S:=Zebrafish:Synaptic%20Pair%20Study/P:=Zebrafish:Image%20Pair%20Study/I:=Zebrafish:Image/B:=(I:Subject)=(Zebrafish:Behavior:Subject)/(I:Step)=(Synapse:Protocol%20Step:ID)/S:Study,I:Subject,B:Learned%3F,S:Region%201%20URL,S:Region%202%20URL
/attribute/S:=Zebrafish:Synaptic%20Pair%20Study/P:=Zebrafish:Image%20Pair%20Study/I:=Zebrafish:Image/B:=(I:Subject)=(Zebrafish:Behavior:Subject)/(I:Step)=(Synapse:Protocol%20Step:ID)/S:Study,I:Subject,B:Learned%3F,S:Region%201%20URL,S:Region%202%20URL


HTTPError: 401 Client Error: Unauthorized for url: https://synapse.isrd.isi.edu/ermrest/catalog/1/attribute/S:=Zebrafish:Synaptic%20Pair%20Study/P:=Zebrafish:Image%20Pair%20Study/I:=Zebrafish:Image/B:=(I:Subject)=(Zebrafish:Behavior:Subject)/(I:Step)=(Synapse:Protocol%20Step:ID)/S:Study,I:Subject,B:Learned%3F,S:Region%201%20URL,S:Region%202%20URL

In [71]:
study_list[0]

{'Learned?': None,
 'Region 1 URL': '/hatrac/Zf/ZfDsy20170412A/SynStd6481-s1-registered.csv:BKWGNDNISI54USDVDSQJSXZTUQ',
 'Region 2 URL': '/hatrac/Zf/ZfDsy20170412A/SynStd6481-s2-registered.csv:4AWIR47HVTPX7Y2EA4RCZCKKBA',
 'Study': 6455,
 'Subject': 'ZfDsy20170412A'}

In [70]:
{
    'PrcDsy20160101A-tpt1': 'aversion',  'PrcDsy20160101A-tpt2': 'aversion',  
    'PrcDsy20170613A-tpt1': 'conditioned', 'PrcDsy20170613A-tpt2' : 'conditioned',
    'PrcDsy20170615A-tpt1': 'unconditioned',  'PrcDsy20170615A-tpt1':'uncondinioned' 
}


{'PrcDsy20160101A-tpt1': 'aversion',
 'PrcDsy20160101A-tpt2': 'aversion',
 'PrcDsy20170613A-tpt1': 'conditioned',
 'PrcDsy20170613A-tpt2': 'conditioned',
 'PrcDsy20170615A-tpt1': 'uncondinioned'}

In [ ]:
synapses = [get_synapses(objectstore, study) for study in study_list]

In [ ]:
plot_study(synapses[0])